In [21]:
import re
import codecs
import pandas as pd
from flask import Flask, request

def decode(input_filepath, output_filepath):
    with codecs.open(input_filepath, "r") as input_file:
        with codecs.open(output_filepath, "w", encoding="utf-8") as output_file:
            for line in input_file:
                file_content = re.sub("\t", ",", line)
                output_file.write(file_content)

decode("pokemon2-8.tsv", "pokemon.csv")

def df_import(csv_file):
    poke_df = pd.read_csv(csv_file, index_col="national_number")
    return poke_df

df_import('pokemon.csv')

app = Flask(__name__)
poke_df = df_import('pokemon.csv')
app.config["db"] = poke_df

@app.route("/pokedex", methods=["GET"])
def pokedex_data():
    """
    HTTP GET: Query for Pokemon 
    """
    global poke_df
    name = request.args.get("name", default=None)
    if name is not None:
        result_df = poke_df.loc[poke_df["english_name"] == name]
    else:
        print("Please enter a valid Pokemon!")
    resp_json = {
        "query_name": name,
        "result": result_df.to_dict(orient="records"),
    }
    resp_headers = {
        "content-type": "application/json",
    }

    return resp_json, 200, resp_headers

if __name__ == '__main__':
    app.run('0.0.0.0', 8080)



 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8080
 * Running on http://172.25.211.59:8080 (Press CTRL+C to quit)
[2022-06-06 14:38:16,910] ERROR in app: Exception on /pokedex [GET]
Traceback (most recent call last):
  File "/home/jessgschu/pokemon_battler/venv/lib/python3.7/site-packages/flask/app.py", line 2077, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/jessgschu/pokemon_battler/venv/lib/python3.7/site-packages/flask/app.py", line 1525, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/jessgschu/pokemon_battler/venv/lib/python3.7/site-packages/flask/app.py", line 1523, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/jessgschu/pokemon_battler/venv/lib/python3.7/site-packages/flask/app.py", line 1509, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "/tmp/ipykernel_22658/661821783.py", line 38, in pokedex_data
    "result": re

Please enter a valid Pokemon!


[2022-06-06 14:38:20,202] ERROR in app: Exception on /pokedex [GET]
Traceback (most recent call last):
  File "/home/jessgschu/pokemon_battler/venv/lib/python3.7/site-packages/flask/app.py", line 2077, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/jessgschu/pokemon_battler/venv/lib/python3.7/site-packages/flask/app.py", line 1525, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/jessgschu/pokemon_battler/venv/lib/python3.7/site-packages/flask/app.py", line 1523, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/jessgschu/pokemon_battler/venv/lib/python3.7/site-packages/flask/app.py", line 1509, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "/tmp/ipykernel_22658/661821783.py", line 38, in pokedex_data
    "result": result_df.to_dict(orient="records"),
UnboundLocalError: local variable 'result_df' referenced before assignment
127.0.0.1 - - [06/Jun/2022 

Please enter a valid Pokemon!
